# SQL Alchemy
Có nhiều RDBMS để quản lý dabase (Oracle, MySQL, SQLite hay thậm chí Microsoft Access), SQL Alchemy cũng có thể coi là một ứng dụng để làm những việc như của RDMS, nhưng đặc biệt hơn là ứng dụng này dùng Python để làm việc đó. SQL Alchemy là tập hợp các database tools bằng Python.

## Giới thiệu SQL Alchemy
### Tác dụng SQL Alchemy 

Được ra mắt năm 2006 và nhanh chóng trở thành một trong các tool ORM (**object-relation mapping**) phổ biến nhất. Rõ ràng SQL không phải dựa trên object-oriented model, mà là relational model, nhưng developer lại thường quen thuộc với tư duy hướng đối tượng hơn. Do đó ORM tools như SQL Alchemy sẽ thu hẹp khoảng cách đó giữa 2 model, 2 cách tư duy, giúp developer vừa có thể làm việc với object vừa có thể thực hiện các thao tác quan trọng với database.

![](./img/ORM.svg)

### Database API

SQLAlchemy có thể work với các loại database cũng như database API khác nhau (DBAPI). **Python Database API Specification** là một chuẩn chung cho Python library (SQLAlchemy,...) để connect đến bất kì database nào, các tiêu chuẩn cụ thể ở trong PEP 249. SQLAlchemy dùng các **dialect** khác nhau để work với các DBAPI khác nhau: MySQL, Oracle, Microsoft SQL Server, PostgreSQL, SQLite, các dialect yêu cầu DBAPI driver thích hợp.

![](./img/DBAPI.svg)

### Setup và làm việc với database có sẵn

Install library bằng các thông thường `pip install sqlalchemy`. Lấy ví dụ một local database tạo bằng SQLite trông như này, file `Building Database.sqlite`:

![](./img/DB.png)

Để connect vào database, ta cần tạo một "engine" và chọn đúng database dialect, cụ thể ở đây là của SQLite (các database còn lại tham khảo ở document của SQLAlchemy [here](https://docs.sqlalchemy.org/en/14/core/engines.html))


In [ ]:

from sqlalchemy import create_engine

engine = create_engine('sqlite:///Buildings Database.sqlite', echo=True')

connection = engine.connect()



Hàm `create_engine` nhận đầu vào là path đến database với dialect của nó, `echo` để in log.

Ngoài ra còn có một số hàm quan trọng khác như `table_names` (in các table), `MetaData`, `Table` (để truy cập thông tin của các table, metadata của Database - **reflection**)



In [ ]:
from sqlalchemy import MetaData, Table

metadata = MetaData()
buildings = Table('Building', metadata, autoload=True, autoload_with=engine)
print(repr(buildings))



Lưu ý là table `'Building'` đã được defined sẵn ở database `Building Database.sqlite`.

## SQL Alchemy Mapping

Có hai loại ORM mapping trong SQLAlchemy: **classical mapping** và **declaring mapping**:

### Classical Mapping

Classical mapping ở đây là cách configuration cho **mapped class** (tức là config cho lớp abstract của table/ database). Ta cần define database table và class Python tương ứng và link lại bằng `mapper()`. Sau đó thay đổi sẽ được cập nhật đến table và class được tạo bởi SQLAlchemy sẽ được lưu xuống database. Classical mapping là một **base mapping system** cung cấp bởi ORM. Ví dụ:


In [ ]:
from sqlalchemy import Table, MetaData, Columm, Integer, String

from sqlalchemy import mapper

metadata = MetaData()

animals = Table('animals', metadata,
           Column('id', Integer, primary_key = True) 
           Column('petname', String(30)),
           Column('age', Integer)
           Column('weight', Integer))

class Animals:
    def __init__(self, petname, age, weight):
        self.petname = petname
        self.age = age 
        self.weight = weight 
        
mapper(Animals, animals)


Đầu tiên chúng ta cần tạo một table để map data từ object Python sang, do đó ta import `Table` và `MetaData` constructor, ngoài ra cần `Column`, `String`, `Integer` để định nghĩa table. `mapper` để link. Dùng biến `animals` là instance của class `Table` để lưu metadata của bảng (tên, các cột, primary key, data type từng cột). 

Bước tiếp theo là định nghĩa class `Animals`, chứa các attribute tương ứng với các cột của table, nếu không sẽ gây lỗi khi mà lưu xuống database.

Cuối cùng là link class `Animals` với `animals` qua `mapper` (trả về mapper object mới ), có nghĩa là chúng ta hoàn toàn có thể chỉnh sửa database bằng Python class và lưu lại. 

### Declaring Mapping

Declarative mapping là phiên bản súc tích hơn của classical mapping, không cần phải tách bạch class và table, tất cả làm trong một class duy nhất.



In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Collumn, String, Integer
from sqlalchemy import create_engine
Base = declarative_base()  # kinda a class so use UpperCamelCase :v 

class Animals(Base)
    __tablename__ = 'animals'

    id = Column(Integer, primary_key = True)

    petname = Column(String(30))
    age = Column(Integer)
    weight = Column(Integer)

engine = create_engine('sqlite://', echo = True)
Base.metadata.create_all(engine)

Code lúc này đã readable hơn, ta đã import `declarative_base` một function để construct một base class (`Base`) để định nghĩa declarative class (class `Animals` lúc này là một declarative class). Bên trong class `Animals`, ta define tên table (bằng `__tablename__`), các cột, data type.

*Lưu ý: Ta không cần phải dùng `__init__` ở đây vì `declarative_base()` đã chứa built-in `__init__` rồi.*

Sau đó ta có thể tạo engine để tiếp tục thao tác trên table, cụ thể là lưu table xuống database bằng `create_all()`

### Sessions

`mapper()` function và declarative extension là **primary configurational interfaces** cho ORM. Sau khi các mapping đã được configured, ta có thể thực hiện **primary usage interface** gọi là session. Session giúp việc giao tiếp với database được thực hiện trơn tru, ta có thể modify database và save changes trong session:

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

animal_1 = Animals(petname='Timmy', age=2, weight=12)
animal_2 = Animals(petnam='Tommy', age=3, weight=10)
animal_3 = Animals(petname='Kitty', age=4, weight=14)

session.add(animal_1)
session.add(animal_2)
session.add(animal_3)
session.commit()

Đầu tiên ta cần define class `Session` với `sessionmaker()`, một configurable **session factory method** mà được bound với engine. Sau đó là set up `session` object bằng cách dùng default constructor `session = Session()`. Từ đó thì session sẽ luôn connect với engine cho đến khi ta commit thay đổi hoặc close session object. `session.add()` để add entry vào trong table. Cuối cùng khi ta muốn lưu thay đổi vào database thì ta thực hiện `session.commit()` để commit transaction, còn không có thể rollback bằng `session.rollback()`.

## SQL Alchemy Querying và Filtering

Sau khi đã biết map class và tạo session, điều quan trọng tiếp theo là cách để *query* và *filter* data từ table.

### Query constructor object

Theo [document](https://docs.sqlalchemy.org/en/14/orm/query.html), một `Query` construction object là nguồn của mọi `SELECT` statement được tạo ra bởi ORM. Tức là `Query` object sẽ tạo một `SELECT` statement cho tất cả các column trong table và rename chúng thành các biến (attribute của Query object) theo PEP convention. Ví dụ như `Query(Animals)` thì sẽ tạo biến `animals_petname` cho cột `petname` của table, tương tự các cột khác. Do đó ta có thể truy cập các value trong các cột tương ứng qua biến này.

Để dùng `Query` thì ta cần một mapped class, để pass như một argument. *Lưu ý: phải là mapped class bởi vì thông thường nếu ta pass một class bình thường thì sẽ raise `sqlalchemy.exc.ArgumentError` exception*

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Collumn, String, Integer
from sqlalchemy.orm import Query  # new import

Base = declarative_base()  # kinda a class so use UpperCamelCase :v 

class Animals(Base)
    __tablename__ = 'animals'

    id = Column(Integer, primary_key = True)

    petname = Column(String(30))
    age = Column(Integer)
    weight = Column(Integer)

query = Query(Animals)

print(query) 
# SELECT animals.id AS animals_id,
# animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight
# FROM animals

### Selecting from the table

Trước khi work với data từ table, ta cần tạo session. Có 2 cách viết cú pháp handle data: Cách 1 là dùng `Query` constructor kèm với session như tham số thứ 2, cách 2 là dùng `with_session()`, cách 3 là dùng `query()` method của session

In [ ]:
from sqlalchemy.orm import sessionmaker, Query 

engine = create_engine("sqlite://:memory:",echo=True)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# first way
query = Query(Animals, session)

# second way
query = Query(Animals)
query = query.with_session(session)

# third way
query = session.query(Animals)

### Retrieving all values

Khi đã có query object, bây giờ ta có thẻ query được các value từ table. Nếu table đã có value (từ ở trên 'Timmy',...) thì ta có thể query tất cả và in ra các thông tin cần thiết như ví dụ dưới đây:

In [ ]:
animal_1 = Animals(petname='Timmy', age=2, weight=12)
animal_2 = Animals(petnam='Tommy', age=3, weight=10)
animal_3 = Animals(petname='Kitty', age=4, weight=14)

session.add(animal_1)
session.add(animal_2)
session.add(animal_3)
session.commit()

all_rows = query.all()

for row in all_rows:
    print(type(row))  # <class '__main__.Animals'>
    print(f'Pet name: {row.petname}, age: {row.age}, weight: {row.weight}')
    # Pet name: Timmy, age: 2, weight: 12
    # Pet name: Tommy, age: 3, weight: 10
    # Pet name: Kitty, age: 4, weight: 14


### Retriving certain values

Ta hoàn toàn có thể query thông tin của một vài cột mà không nhất thiết phải query hết tất cả (`Query.all()`) bằng các specify các cột cần query trong lúc query:

In [ ]:
query = session.query(Animals.petname, Animals.age)
print(query)  # SELECT animals.petname AS animals_petnam, animals.age AS animals_age FROM animals

for name, age in query:
    print(name, age)
    # Timmy 2
    # Tommy 3
    # Kitty 4

Ngoài ra còn có method `count()` cũng rất hữu ích

In [ ]:
query = session.query(Animals)
print(query.count())  # 3

### Filtering the table

Để filter data (kiểu giống như WHERE trong SQL), ta cần tham số để filter. Ví dụ ta muốn tìm tuổi và cân nặng của animal với tên là "Tommy":

In [ ]:

query = session.query(Animals)

for row in query.filter(Animals.petname == 'Tommy'):
    print(row.petname, row.age, row.weight)  # Tommy 3 10
# SELECT animals.id AS animals_id, animals.petname AS animals_petname, 
# animals.age AS animals_age, animals.weight AS animals_weight 
# FROM animals
# WHERE animals.petname = "Tommy"

Các tham số của filter là các boolean expression nên hoàn toàn không có gì quá xa lạ hay bất kì giới hạn đặc biệt nào:

In [ ]:
query = session.query(Animals.age, Animals.weight)

age_gr_than = Animals.age > 2
weight_eq_gr_than = Animals.weight >= 11

for age, weight in query.filter(age_gr_than, weight_eq_gr_than):
    print(f"Pet age: {age}, Pet weight: {weight}")  # Pet age: 4, Pet weight: 14